In [11]:
import sqlite3
import pandas as pd

db_path = "database.sqlite"

conn = sqlite3.connect(db_path)

tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
tables

,name
0,sqlite_sequence
1,Player_Attributes
2,Player
3,Match
4,League
5,Country
6,Team
7,Team_Attributes


In [12]:
import pandas as pd

match = pd.read_sql("SELECT * FROM Match", conn)
team = pd.read_sql("SELECT * FROM Team", conn)
team_attr = pd.read_sql("SELECT * FROM Team_Attributes", conn)

print(match.shape, team.shape, team_attr.shape)

(25979, 115) (299, 5) (1458, 25)


In [13]:
df = match.copy()

def outcome(row):
    hg = row["home_team_goal"]
    ag = row["away_team_goal"]
    if hg > ag:
        return "H"
    if hg < ag:
        return "A"
    return "D"

df["target_outcome"] = df.apply(outcome, axis=1)

leak_cols = [
    "home_team_goal", "away_team_goal",
    "goal", "shoton", "shotoff", "foulcommit",
    "card", "cross", "corner", "possession"
]

for c in leak_cols:
    if c in df.columns:
        df = df.drop(columns=[c])

df = df.dropna(subset=["home_team_api_id", "away_team_api_id", "season"])

df[["season","home_team_api_id","away_team_api_id","target_outcome"]].head(), df.shape


(      season  home_team_api_id  away_team_api_id target_outcome
 0  2008/2009              9987              9993              D
 1  2008/2009             10000              9994              D
 2  2008/2009              9984              8635              A
 3  2008/2009              9991              9998              H
 4  2008/2009              7947              9985              A,
 (25979, 106))

In [14]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("database.sqlite")

team_attributes = pd.read_sql("SELECT * FROM Team_Attributes", conn)

team_attributes.shape


(1458, 25)

In [16]:
import pandas as pd

# kopija Team_Attributes
ta = team_attributes.copy()
ta["date"] = pd.to_datetime(ta["date"])
ta = ta.sort_values("date")

# numeričke kolone, bez id i team_api_id
num_cols = ta.select_dtypes(include="number").columns.tolist()
num_cols = [c for c in num_cols if c not in ["id", "team_api_id"]]

ta = ta[["team_api_id", "date"] + num_cols]

def attach_team_attrs(match_df, team_df, side):
    left = match_df.sort_values("date")
    right = team_df.sort_values("date")

    merged = pd.merge_asof(
        left,
        right,
        left_on="date",
        right_on="date",
        left_by=f"{side}_team_api_id",
        right_by="team_api_id",
        direction="backward"
    )

    merged = merged.drop(columns=["team_api_id"])

    # prefiksiraj SAMO team attribute kolone (ne date!)
    for c in num_cols:
        merged = merged.rename(columns={c: f"{side}_{c}"})

    return merged

# osiguraj datetime u match df
df["date"] = pd.to_datetime(df["date"])

# home pa away
df2 = attach_team_attrs(df, ta, "home")
df3 = attach_team_attrs(df2, ta, "away")

df3.shape


(25979, 126)

In [ ]:
# 1) izbaci ID/URL kolone koje ne pomažu modelu i samo kompliciraju
drop_like = [
    "id", "match_api_id", "match_id",
    "home_team_api_id", "away_team_api_id",  # ovo bi moglo ostati, ali bolje izbaciti radi previše kategorija
    "home_team_fifa_api_id", "away_team_fifa_api_id",
    "stage", "match_api_id", "league_id", "country_id"
]

for c in drop_like:
    if c in df3.columns:
        df3 = df3.drop(columns=[c])

# 2) makni sve "object" kolone osim sezone (sezonu ćemo ostaviti kao kategoriju)
obj_cols = df3.select_dtypes(include="object").columns.tolist()
keep_obj = ["season", "target_outcome"]
for c in obj_cols:
    if c not in keep_obj:
        df3 = df3.drop(columns=[c])

# 3) NaN: za numeričke kolone zamijeni medianom (jednostavno, stabilno)
num_cols2 = df3.select_dtypes(include="number").columns.tolist()
for c in num_cols2:
    df3[c] = df3[c].fillna(df3[c].median())

# 4) makni redove gdje target fali (ne bi smjelo)
df3 = df3.dropna(subset=["target_outcome"])

# 5) export
final_path = "matches_final.csv"
df3.to_csv(final_path, index=False)

df3.shape, final_path
